¡Es genial que quieras subir tus primeros proyectos de Data Science a GitHub! Una buena estructura y formato hacen que tu proyecto sea más profesional, fácil de entender y colaborativo. Aquí tienes algunas sugerencias:

---

### **1. Estructura Recomendada para el Repositorio**
Puedes organizar tu proyecto así:

```
nombre-del-proyecto/
│
├── data/                   # Datos (crudos y procesados)
│   ├── raw/                 # Datos originales (sin modificar)
│   ├── processed/           # Datos limpios/transformados
│   └── external/            # Datos de fuentes externas (opcional)
│
├── notebooks/               # Jupyter Notebooks (.ipynb)
│   ├── 01-exploracion.ipynb # Ej: Análisis exploratorio
│   └── 02-modelo.ipynb      # Ej: Entrenamiento del modelo
│
├── src/                     # Código en scripts reutilizables (.py o .R)
│   ├── preprocessing.py     # Funciones de limpieza
│   └── modeling.py          # Funciones de modelado
│
├── reports/                 # Resultados (gráficos, informes)
│   ├── figures/             # Imágenes/gráficos
│   └── resultados.pdf       # Conclusiones (opcional)
│
├── README.md                # Explicación del proyecto
└── requirements.txt         # Dependencias (Python)
```

---

### **2. Formato del Notebook (Jupyter)**
- **Nombre descriptivo**: Ej: `01-data-cleaning.ipynb`, `02-random-forest.ipynb`.
- **Encabezado claro**: Incluye un título, autor, fecha y descripción breve.
- **Celdas Markdown**: Explica el propósito de cada sección.
- **Comentarios en el código**: Sé claro, pero no redundante.
- **Resultados visibles**: Guarda los outputs (gráficos, tablas) para que se vean sin ejecutar.
- **Limpieza**: Elimina celdas de prueba o código innecesario.

Ejemplo de estructura dentro del notebook:
```markdown
# Análisis de Datos: Titanic
**Autor**: Tu Nombre  
**Fecha**: Julio 2024  

## 1. Carga de Datos  
Descripción breve...

## 2. Limpieza  
- Manejo de valores nulos...
```

---

### **3. Buenas Prácticas para GitHub**
- **README.md**: Explica:
  - Objetivo del proyecto.
  - Dataset usado (fuente, variables clave).
  - Pasos para reproducirlo (ej: `pip install -r requirements.txt`).
  - Resultados destacados.
- **.gitignore**: Excluye archivos grandes o sensibles (ej: `.ipynb_checkpoints/`, `/data/` si los datos son pesados).
- **Licencia**: Agrega una (ej: MIT, Apache) si es público.
- **Activa GitHub Pages**: Opcional, para mostrar resultados como un portfolio.

---

### **4. Extras Profesionales**
- **Exporta a HTML/PDF**: Para una lectura fácil sin ejecutar el notebook.
  ```bash
  jupyter nbconvert --to html mi_notebook.ipynb
  ```
- **Documentación**: Si el proyecto es complejo, agrega un `docs/` con explicaciones detalladas.
- **CI/CD**: Configura tests automáticos (con GitHub Actions) si usas scripts.

---

### **Ejemplo de Repositorio Inspirador**
Revisa proyectos como:
- [Cookiecutter Data Science](https://drivendata.github.io/cookiecutter-data-science/) (plantilla estándar).
- [Titanic: Machine Learning from Disaster](https://github.com/agconti/kaggle-titanic) (ejemplo sencillo).

¡Sube tu proyecto y compártelo con la comunidad! Si tienes dudas específicas, dime y te ayudo. 🚀

# %% [markdown]
"""
# Data Wrangling: Dataset de Ventas (Ejemplo con Errores Comunes)
**Autor**: Tu Nombre  
**Fecha**: Julio 2024  

Descripción:  
Este notebook limpia y transforma un dataset de ventas con múltiples problemas típicos:
- Valores nulos inconsistentes.
- Formatos de fecha y texto inconsistentes.
- Outliers absurdos.
- Columnas redundantes o inútiles.
"""
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# %% [markdown]
## 1. Carga de Datos
Dataset ficticio `sales_dirty.csv` con los siguientes problemas:
- `id`: Duplicados.
- `fecha`: Mix de formatos (dd/mm/yyyy y mm/dd/yyyy).
- `producto`: Nombres inconsistentes ("Laptop" vs "laptop" vs "LAPTOP").
- `precio`: Outliers (ej: 1 millón por un lápiz), strings con "$".
- `cantidad`: Negativos y nulos.
- `cliente_id`: Mix de números y strings ("001" vs 1).
"""
# %%
df = pd.read_csv("data/raw/sales_dirty.csv")
df.head(10)

# %% [markdown]
## 2. Exploración Inicial
# %%
# Info general
print("Filas y columnas:", df.shape)
print("\nTipos de datos:\n", df.dtypes)
print("\nValores nulos:\n", df.isnull().sum())

# Resumen estadístico (¡cuidado con los outliers!)
df.describe(include="all")

# %% [markdown]
## 3. Limpieza Paso a Paso

### 3.1. Eliminar Duplicados
# %%
print("Filas antes:", len(df))
df.drop_duplicates(subset="id", keep="first", inplace=True)
print("Filas después:", len(df))

### 3.2. Normalizar Fechas
# %%
# Problema: Mix de formatos (ej: "15/01/2023" vs "01/20/2023" -> mes 20?)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")  # Convierte a fecha y devuelve NaT si falla

# Verificar fechas inválidas
print("Fechas inválidas:\n", df[df["fecha"].isna()])

### 3.3. Normalizar Texto (Productos)
# %%
df["producto"] = df["producto"].str.lower().str.strip()  # Todo a minúsculas y sin espacios

### 3.4. Limpiar Precios
# %%
# Eliminar "$" y convertir a float
df["precio"] = df["precio"].replace('[\$,]', '', regex=True).astype(float)

# Identificar outliers (ej: precio > 1000)
outliers = df[df["precio"] > 1000]
print("Outliers de precio:\n", outliers)

# Reemplazar outliers con la mediana
median_price = df["precio"].median()
df["precio"] = np.where(df["precio"] > 1000, median_price, df["precio"])

### 3.5. Limpiar Cantidad
# %%
# Eliminar negativos y reemplazar nulos con 0
df["cantidad"] = df["cantidad"].clip(lower=0).fillna(0)

### 3.6. Normalizar IDs de Clientes
# %%
# Convertir todo a string y rellenar ceros (ej: "1" -> "001")
df["cliente_id"] = df["cliente_id"].astype(str).str.zfill(3)

# %% [markdown]
## 4. Transformaciones Adicionales

### 4.1. Crear Columna "total_venta"
# %%
df["total_venta"] = df["precio"] * df["cantidad"]

### 4.2. Eliminar Columnas Inútiles
# %%
df.drop(columns=["notas", "codigo_interno"], inplace=True, errors="ignore")

# %% [markdown]
## 5. Resultado Final
# %%
# Dataset limpio
print("Valores nulos restantes:\n", df.isnull().sum())
df.head()

# Guardar datos procesados
df.to_csv("data/processed/sales_clean.csv", index=False)

# %% [markdown]
## 6. Visualización de Cambios (Opcional)
# %%
# Antes vs Después (ej: distribución de precios)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.title("Precios (Original)")
plt.hist(pd.to_numeric(df_original["precio"].replace('[\$,]', '', regex=True), errors="coerce"), bins=20)
plt.subplot(1, 2, 2)
plt.title("Precios (Limpio)")
plt.hist(df["precio"], bins=20)
plt.show()